In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import fcluster

from sklearn.metrics import silhouette_score


In [0]:
# 계층적 군집

# 군집 트리, 덴드로그램을 생성하여 다양한 데이터를 그룹화
# 비슷한 군집끼리 묶어 가면서 최종적으로 하나의 군집단으로 묶는 기법

# 즉, 군집간의 거리를 기반으로 군집화 하는 방식으로 
# 기존의 군집기법(kmeans)에 비해 군집수를 지정X

# 계층적 군집 => 응집형, 분리형

# 응집형 : 개별 데이터 군집 => 군집단 형성
# 분리형 : 데이터 전체를 하나의 군집 => 세부적으로 여러 군집으로 나눔


X = np.array([ [5,3], [10,15], [15,12], [24,10], [30,30], 
               [85,70], [71,80], [60,78], [70,55], [80,91] ])

plt.grid(True)
plt.plot(X[:, 0], X[:, 1], 'ro')
plt.show()



# 덴드로그램 시각화
# linkage 함수의 method 속성으로 각 군집간 연결방식을 지정
# => single / average / complete / centeroid / ward

# 단일 single    : 각 군집을 대상으로 최소거리 기준
# 평균 average   : 각 군집을 대상으로 평균거리 기준
# 최대 complete  : 각 군집을 대상으로 최대거리 기준
# 중심 centeroid : 각 군집 내 중심점 기준
# 와드 ward      : 각 군집간 SSW, SSB 간의 차이를 이용


linked = linkage(X, method = 'ward')   # 거리계산법 지정

dendrogram(linked, 
           orientation='top',
           labels=range(1, 11),
           distance_sort='descending', 
           show_leaf_counts=True)

plt.xlabel('species')   # 
plt.ylabel('distance')   # 군집간 거리

plt.axhline(20, color='red', linestyle='dashed')
# 군집 간 거리가 20일때 6개의 군집으로 나뉨

plt.axhline(30, color='red', linestyle='dashed')
# 군집 간 거리가 30일때 4개의 군집으로 나뉨

plt.show()



# 덴드로그램을 보고 n개의 군집으로 나눌려고 할 때,
# 적절한 distance는 어떻게 알아볼까?

# => 덴드로그램에서 밑에서 위로 올라갈수록
#    군집을 의미하는 선의 갯수가 줄어듬(응집형 군집!!)

# scipy 에서는 fcluster 함수를 이용해서 
# distance가 특정값 일 때 군집정보를 알려줌
# fcluster(연결방식, 거리, 거리측정단위)

clusters = fcluster(linked, 20, criterion='distance')

nc = np.unique(clusters)
np.count_nonzero(nc)


clusters = fcluster(linked, 30, criterion='distance')

nc = np.unique(clusters)
np.count_nonzero(nc)



# sklearn 을 이용해서 응집형 군집 적용
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=2, 
                                  affinity='euclidean', 
                                  linkage='ward')
cluster.fit_predict(X)
print(cluster.labels_)



# 군집된 결과 시각화
plt.scatter(X[:, 0], X[:, 1], c=cluster.labels_, cmap='rainbow')
plt.grid(True)
plt.show()



# 군집 결과 평가하기
import kmeans_eval

silhouette_score(X, cluster.labels_)

kmeans_eval.visualize_silhouette([2, 3, 4, 5], X)


In [0]:
# iris 데이터셋을 이용해서 계층적 군집화 시도
from sklearn.datasets import load_iris

# 데이터 적재
iris = load_iris()

X = iris.data
y = iris.target


# 군집간 연결방식 지정 후 덴드로그램 작성
linked = linkage(X, method='average')

plt.figure(figsize=(18, 10))   # 1800 x 1000
dendrogram(linked, labels=y, 
           leaf_rotation=0, leaf_font_size=12)

plt.show()



# 군집을 3개로 나눌려고 할 때 적절한 distance는?
iris_hc = fcluster(linked, 2, criterion='distance')
cm = pd.crosstab(iris_hc, y)
print(cm)


iris_hc = fcluster(linked, 1.85, criterion='distance')
cm = pd.crosstab(iris_hc, y)
print(cm)   # 군집의 수가 2개


iris_hc = fcluster(linked, 1.79, criterion='distance')
cm = pd.crosstab(iris_hc, y)
print(cm)   # 군집의 수가 3개이지만, 잘못 분류된 요소 14개 존재



# sklearn 으로 응집형 군집화 시도
cluster = AgglomerativeClustering(
            n_clusters=3, affinity='euclidean', linkage='ward')

cluster.fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=cluster.labels_, cmap='rainbow')
plt.grid(True)
plt.show()

plt.scatter(X[:, 2], X[:, 3], c=cluster.labels_, cmap='rainbow')
plt.grid(True)
plt.show()
